In [1]:
from BinanceBackTest import *
import asyncio
import nest_asyncio
import os
import utils
from pprint import pprint
import Analysis
import DataProcess

nest_asyncio.apply()

In [2]:
symbols = ['BTCUSDT', 'XRPUSDT', 'ADAUSDT']
intervals = ['1m', '3m', '5m', '1h']
start_date = '2024-5-1 00:00:00'
end_date = '2024-11-29 23:59:59'

obj_process = DataProcess.TradeStopper()
obj_analy = Analysis.AnalysisManager(back_test=True)
obj_analy.intervals = intervals
obj_order = OrderManager()
obj_con = DataProcess.OrderConstraint()
obj_data = DataManager(symbols=symbols,
                       intervals=intervals,
                       start_date=start_date,
                       end_date=end_date)

data_length, real_time_data = asyncio.run(obj_data.read_data_run(download_if_missing=False))#(download_if_missing=get_futures_kilne_datarue))

In [ ]:

trade_data = []


seed_money = 100

obj_wallet = WalletManager(initial_fund=seed_money)

n = 0

for i in range(10, data_length, 1):
    data = obj_data.get_real_time_segments(end_idx=i, real_time_kilne=real_time_data)
    for symbol in symbols:
        
        # 시나리오 1 검사.
        case_1 = obj_analy.scenario_1(symbol=symbol, convert_data=data)
        
        # 주문 제약 사항 스펙
        constraint = obj_con.calc_fund(obj_wallet.balance_info['total_assets'], rate=0.2)
        # print(constraint)
        trade_count = constraint.get('count')
        trade_balance = constraint.get('tradeValue')
        # print(trade_balance)
        # print(constraint)
        # 시나리오 1 조건.
        if case_1[1] >0 and case_1[2] >=3 and case_1[3] >=3 and case_1[-1]:
            status, order_signal_open = asyncio.run(obj_order.generate_order_open_signal(symbol=symbol,
                                                                                         position=case_1[1],
                                                                                         leverage=int(case_1[4]/60),
                                                                                         balance=trade_balance,
                                                                                         entry_price=data.get(symbol).get('1m')[-1][4],
                                                                                         open_timestamp = data.get(symbol).get('1m')[-1][6]))
            # 주문 신호 발생시
            if status and not isinstance(obj_wallet.account_balances.get(symbol), dict) and trade_count > obj_wallet.balance_info['locked_funds']['number_of_stocks']:
                # 포지션 종료를 위한 초기값 업데이트
                obj_process.initialize_trading_data(symbol=symbol,
                                                    position=order_signal_open['position'],
                                                    entry_price =order_signal_open['entryPrice'])
                # 지갑 정보 업데이트
                obj_wallet.add_funds(order_signal=order_signal_open)
                n += 1
                # print(obj_wallet.account_balances)

                # # DEBUG PRINT
                # print(order_signal_open)
                # print(obj_wallet.balance_info)
        
        # 포지션 종료 검사.
        # 현재가 
        current_price = float(data[symbol]['1m'][-1][4])
        # wallet 보유현황
        is_wallet = obj_process.trading_data.get(symbol)
        # print(is_wallet)
        if is_wallet is not None:
            is_stop = obj_process.get_trading_stop_signal(symbol=symbol,
                                                          current_price=current_price)
            if is_stop:
                value = obj_order.generate_order_close_signal(symbol=symbol,
                                                              current_price=current_price,
                                                              wallet_data=obj_wallet.account_balances)
                obj_wallet.remove_funds(symbol=symbol)
                obj_process.remove_trading_data(symbol)
                # n + 1

    obj_wallet.get_wallet_status(symbol=symbol, current_price=current_price)
    utils._std_print(obj_wallet.balance_info['total_assets'])

6.9239200000000003

KeyboardInterrupt: 

In [ ]:
obj_wallet.account_balances

In [ ]:
obj_wallet.balance_info

In [ ]:
96.46296+2.56984

In [ ]:
obj_wallet.account_balances